# Toy example

To check that everything is working well, we will replicate an example from Murphy (2022). The Python code generating the example has been [open sourced](https://github.com/probml/pyprobml/blob/master/scripts/logreg_laplace_demo.py). The author of the code uses the `bayes_logistic` function to run Bayesian Logistic Regression (BLR) using Laplace approximation (source code available [here](https://bayes-logistic.readthedocs.io/en/latest/_modules/bayes_logistic/bayes_logistic.html#fit_bayes_logistic)). Here we will code up BLR from scratch in Julia programming language. First, let's generate the synthetic dataset used for the example in Murphy (2022).

In [1]:
;python toy_data.py

In [2]:
X

LoadError: UndefVarError: X not defined